# Language model in character level 

## Preprocessing of data

In [1]:
import numpy as np
import pandas as pd
import nltk, itertools, re
import itertools, operator
from collections import Counter

# data = open('data/reddit-comments-2015-08.csv', 'r').read()

df = pd.read_csv('data/reddit-comments-2015-08.csv')

# with open('data/reddit-comments-2015-08.csv', 'r') as f:
#     reader = 


In [2]:

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()



In [3]:
#if read from a plain text file
data = open('data/text.txt', 'r').read()
data


'I joined a new league this year and they have\n'

#### Char-level data processing

In [4]:
char_data = [clean_str(s.strip()) for s in df.body]
char_data = '.'.join(char_data)

# #Consider char-level
# char_list = "abcdefghijklmnopqrstuvwxyz0123456789 ,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"

chars = list(set(char_data))
chars=sorted(chars)
print(chars)
print("Data has %d characters in which %d are unique" %(len(char_data), len(chars)))


[' ', '!', "'", '(', ')', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '\\', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Data has 7500269 characters in which 46 are unique


In [5]:
# create char-to-index, index-to-char lists
unknown_char = '§'
index_to_char = [unknown_char] +[x[0] for x in chars] 
char_to_index = {c:i for i, c in enumerate(index_to_char)} #{c:i for i, c in enumerate(chars)}
print(char_to_index, "\n", index_to_char)


{'§': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '.': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '8': 16, '9': 17, '?': 18, '\\': 19, '`': 20, 'a': 21, 'b': 22, 'c': 23, 'd': 24, 'e': 25, 'f': 26, 'g': 27, 'h': 28, 'i': 29, 'j': 30, 'k': 31, 'l': 32, 'm': 33, 'n': 34, 'o': 35, 'p': 36, 'q': 37, 'r': 38, 's': 39, 't': 40, 'u': 41, 'v': 42, 'w': 43, 'x': 44, 'y': 45, 'z': 46} 
 ['§', ' ', '!', "'", '(', ')', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '\\', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


#### Word-level data processing

In [6]:
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

#Consider word-level
# word_data = ["%s %s %s" %(sentence_start_token, s, sentence_end_token) for s in df.body]
# word_data[0]

def getTokens(lines, min_sent_characters=1):
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x) for x in lines])
    sentences = [clean_str(s) for s in sentences if "http" not in s and len(s) >= min_sent_characters]
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    return sentences

sents = getTokens(df.body)


In [7]:
vocabulary_size = 8000

def buildVocab(sentences):
    # Tokenize the sentences into words
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

#     # Build vocabulary
#     word_counts = Counter(itertools.chain(*tokenized_sentences))
#     print("Found %d unique words tokens." % len(word_counts.items()))
#     print("Using vocabulary size %d." % vocabulary_size)
#     print(word_counts)
 
#     # Mapping from index to word
#     index_to_word =  ["<MASK/>", unknown_token] + [x[0] for x in word_counts.most_common(vocabulary_size)]
#     # Mapping from word to index
#     word_to_index = {x: i for i, x in enumerate(index_to_word)}
    
    
    ####### If we want to order the words in alphabet #######
    # Count the word frequencies
    word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    print("Found %d unique words tokens." % len(word_freq.items()))

    # Get the most common words and build index_to_word and word_to_index vectors
    vocab = sorted(word_freq.items(), key=lambda x: (x[1], x[0]), reverse=True)[:vocabulary_size-2]
    print("Using vocabulary size %d." % vocabulary_size)
    print("The most frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[0][0], vocab[0][1]))
    print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
 
    sorted_vocab = sorted(vocab, key=operator.itemgetter(1))
    index_to_word = ["<MASK/>", unknown_token] + [x[0] for x in sorted_vocab]
    word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])
    
    
    print(word_to_index)
    return index_to_word, word_to_index
    
index_to_word, word_to_index = buildVocab(sents)

Found 47970 unique words tokens.
Using vocabulary size 8000.
The most frequent word in our vocabulary is 'SENTENCE_START' and appeared 77704 times.
The least frequent word in our vocabulary is 'metabolism' and appeared 9 times.
{'<MASK/>': 0, 'UNKNOWN_TOKEN': 1, 'zelda': 2, 'zealand': 3, 'yourselves': 4, 'xanax': 5, 'wrecked': 6, 'worn': 7, 'worms': 8, 'wolfenstein': 9, 'witnessed': 10, 'witness': 11, 'wished': 12, 'wipes': 13, 'wilderness': 14, 'whoops': 15, 'whim': 16, 'weddings': 17, 'wax': 18, 'wasteful': 19, 'washed': 20, 'warped': 21, 'warn': 22, 'vw': 23, 'vpn': 24, 'vous': 25, 'vog': 26, 'vocals': 27, 'vlad': 28, 'vita': 29, 'visuals': 30, 'visually': 31, 'vim': 32, 'vil': 33, 'verses': 34, 'vengeance': 35, 'vapor': 36, 'vader': 37, 'vac': 38, 'uterus': 39, 'upfront': 40, 'unstoppable': 41, 'unreliable': 42, 'unpleasant': 43, 'unjust': 44, 'unity': 45, 'unclear': 46, 'uhc': 47, 'tywin': 48, 'trends': 49, 'transaction': 50, 'traits': 51, 'tragic': 52, 'tragedy': 53, 'tourist': 5

## Char-level Model

In [8]:
hidden_size = 200 # nb of neurons in hidden layer
seq_length = 25 # number of steps to unroll the RNN for, and this is also nb of chars putting in the input of RNN
learning_rate = 1e-1

data_size = len(char_data)
vocab_size = len(chars) 

#init parameters
W_hh = np.random.randn(hidden_size, hidden_size) #(H,H)
W_xh = np.random.randn(vocab_size, hidden_size) #(D,H)
W_hy = np.random.randn(hidden_size, vocab_size) #(H,M), M=D
b_h = np.zeros((1,hidden_size)) #(H,)
b_y = np.zeros((1,vocab_size)) #(M,)


## Layers

In [9]:
#Reference from @Karpathy: https://gist.github.com/karpathy/d4dee566867f8291f086
#
#                         [b_h]                                                [b_y]
#    w2v                    v                                (h_next)            v
#  x --> x_s -> [W_xh] -> [sum] -> h_raw -> [nonlinearity] -> h_s -> [W_hy] -> [sum] -> y_s -> [exp(y[k])/sum(exp(y))] -> p_s
#                           ^                                  |
#                           '----h_prev------[W_hh]------------'
#


### RNN Forward

In [10]:
def rnn_forward_single_step(x, h_prev, W_xh, W_hh, W_hy, b_h, b_y):
    """
    Run the forward pass for a single timestep of a vanilla RNN that uses a tanh
    activation function.

    The input data has dimension D, the hidden state has dimension H, and we use
    a minibatch size of N.

    Inputs:
    - x: Input data for this timestep, of shape (N, D).
    - h_prev: Hidden state from previous timestep, of shape (N, H)
    - W_xh: Weight matrix for input-to-hidden connections, of shape (D, H)
    - W_hh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - W_hy: Weight matrix for hidden-to-output connections, of shape (H, M)
    - b_h: Biases of shape (H,)
    - b_y: Bias of shape (M, )

    Returns a tuple of:
    - h_next: Next hidden state, of shape (N, H)
    - y_s: output of this timestep (N, M)
    - cache: Tuple of values needed for the backward pass.
    """
    
    h_raw = np.dot(x, W_xh) + np.dot(h_prev, W_hh) + b_h  #(N,D)x(D,H) + (N,H)x(H,H) +(1,H) = (N,H)
    h_next = np.tanh(h_raw) #(N, H)
    y_s = np.dot(h_next, W_hy) + b_y #(N, H)x(H,M) +(1,M) =(N,M)
#     p_s = np.exp(y_s) / np.sum(np.exp(y_s), axis=0)
    
    cache = (x, h_prev, h_next, W_xh, W_hh, W_hy, b_h, b_y)
    return h_next, y_s, cache


def rnn_forward(x, h0, W_xh, W_hh, W_hy, b_h, b_y):
    """
    Run a vanilla RNN forward on an entire sequence of data. We assume an input
    sequence composed of T vectors (each vector represents a word/char), each of dimension D. 
    The RNN uses a hidden
    size of H, and we work over a minibatch containing N sequences. After running
    the RNN forward, we return the hidden states for all timesteps.
    Inputs:
    - x: Input data for the entire timeseries, of shape (N, T, D).
    - h0: Initial hidden state, of shape (N, H)
    - W_xh: Weight matrix for input-to-hidden connections, of shape (D, H)
    - W_hh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - W_hy: Weight matrix for hidden-to-output connections, of shape (H, M)
    - b_h: Biases of shape (H,)
    - b_y: Bias of shape (M, )
    Returns a tuple of:
    - h: Hidden states for the entire timeseries, of shape (N, T, H)
    - y: Output of the entire timeseries, of shape (N, T, M)
    - cache: Values needed in the backward pass
    """
    N, T, D = x.shape
    H, M = W_hy.shape
    
    h = np.empty((N, T, H))
    cache = {}
    y_s = np.empty((N, T, M))
    
    for i in range(T):
        if i==0: 
            h[:, i, :], y_s[:, i, :], cache[i] = rnn_forward_single_step(x, h0, W_xh, W_hh, W_hy, b_h, b_y)
        else: 
            h[:, i, :], y_s[:, i, :], cache[i] = rnn_forward_single_step(x, h[:, i-1, :], W_xh, W_hh, W_hy, b_h, b_y)
    
    return h, y_s, cache




### RNN Backward

In [11]:
#Reference from @Karpathy: https://gist.github.com/karpathy/d4dee566867f8291f086
#
#                         [b_h] (1,H)                                         [b_y] (1,M)
#    w2v                    v      (N,H)                     (N,H)   (H,M)       v
#  x --> x_s -> [W_xh] -> [sum] -> h_raw -> [nonlinearity] -> h_s -> [W_hy] -> [sum] -> y_s -> [exp(y[k])/sum(exp(y))] -> p_s
# (N,D)         (D,H)       ^                                  |                       (N,M)
#                           '----h_prev------[W_hh]------------'
#                                (N,H)       (H,H)

def rnn_backward_single_step (dh_next, dy, cache):
    """
    Backward pass for a single timestep of a vanilla RNN.
    Inputs:
    - dh_next: Gradient of loss with respect to next hidden state (N, H)
    - dy: of shape (N,M)
    - cache: Cache object from the forward pass
    Returns a tuple of:
    - dx: Gradients of input data, of shape (N, D)
    - dh_prev: Gradients of previous hidden state, of shape (N, H)
    - dWxh: Gradients of input-to-hidden weights, of shape (D, H)
    - dWhh: Gradients of hidden-to-hidden weights, of shape (H, H)
    - dWhy: Gradients of hidden-to-output weights, of shape (H, M)
    - dbh: Gradients of bias vector, of shape (H,)
    - dby: Gradients of bias vector, of shape (M,)
    
    """
    dx, dh_prev, dWxh, dWhh, dWhy, dbh, dby = None, None, None, None, None, None, None
    x, h_prev, h_next, W_xh, W_hh, W_hy, b_h, b_y = cache
    
    
    
    dby = np.sum(dy, axis=0) #(1,M)
    dWhy = np.dot(h_next.T, dy) #(H,N)x(N,M) = (H,M)
    dh = np.dot(dy, Why.T) + dh_next # backprop into h, (N,M)x(M,H)=(N,H)
    dh_raw = (1 - h_next ** 2) * dh # note: tanh(x)' = 1 - tanh^2(x), shape=(N,H)
    dbh = np.sum(dh_raw, axis=0) #(1,H)
    dWxh = np.dot(x.T, dh_raw) # (D,N)x(N,H) = (D,H)
    dWhh = np.dot(h_prev.T, dh_raw) # (H,N)x(N,H)=(H,H)
    dx = np.dot(dh_raw, W_xh.T) # (N,H)x(H,D) = (N,D)
    dh_prev = np.dot(dh_raw, W_hh.T) # (N,H)x(H,H) = (N,H)
    
    ########
#     dpre_actv = (1 - next_h ** 2) * dnext_h         # (N, H)
#     dx = dpre_actv.dot(Wx.T)
#     dprev_h = dpre_actv.dot(Wh.T) #(N,H)x(H,H)=(N,H)
#     dWx = x.T.dot(dpre_actv)
#     dWh = prev_h.T.dot(dpre_actv)
#     db = np.sum(dpre_actv, 0)
    
    
#     dWhy += np.dot(dy, hs[t].T)
#     dby += dy
#     dh = np.dot(Why.T, dy) + dhnext # backprop into h
#     dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
#     dbh += dhraw
#     dWxh += np.dot(dhraw, xs[t].T)
#     dWhh += np.dot(dhraw, hs[t-1].T)
#     dhnext = np.dot(Whh.T, dhraw)
    
    return dx, dh_prev, dWxh, dWhh, dWhy, dbh, dby


def rnn_backward(dh, dy, cache):
    """
    Compute the backward pass for a vanilla RNN over an entire sequence of data.
    Inputs:
    - dh: Upstream gradients of all hidden states, of shape (N, T, H)
    - dy: Upstream gradients of output, of shape (N, M)
    - cache
    Returns a tuple of:
    - dx: Gradient of inputs, of shape (N, T, D)
    - dh0: Gradient of initial hidden state, of shape (N, H)
    - dWx: Gradient of input-to-hidden weights, of shape (D, H)
    - dWh: Gradient of hidden-to-hidden weights, of shape (H, H)
    - dbh: Gradient of biases, of shape (H,)
    - dby: Gradient of biases, of shape (M,)
    """
    N, T, H =dh.shape
    N, M = dy.shape
    
    
    dx = np.empty((N,T,D))
    dWxh = np.zeros((D,H))
    dWhh = np.zeros((H,H))
    dWhy = np.zeros((H,M))
    dbh = np.zeros((1,H))
    dby = np.zeros((1,M))
    dh_prev = 0
    
    for i in reversed(range(T)):
        # Add the current timestep upstream gradient to previous calculated dh
        dh_next = dh_prev + dh[:,i,:]
        dx[:,i,:], dh_prev, dWxh_temp, dWhh_temp, dWhy_temp, dbh_temp, dby_temp = \
                rnn_backward_single_step(dh_next, dy[:,i,:], cache[i])
        dWxh += dWxh_temp
        dWhy += dWhy_temp
        dWhh += dWhh_temp
        dbh += dbh_temp
        dby += dby_temp
    
    dh0 = dh_prev
    return dx, dh0, dWxh, dWhh, dWhy, dbh, dby
    

### Affine forward/backward

In [14]:
## Affine forward
def affine_forward(x, W, b):
    """
    Inputs:
    x - Input data, of shape (N, D) #may be generally (N,d1,...d_k) 
    w - Weights, of shape (D, M)
    b - Biases, of shape (M,)

    Returns a tuple of:
    - out: output, of shape (N, M)
    - cache: (x, w, b)
    """
    
    out = np.dot(np.reshape(x,(N,-1)), W) + b #out = xW+b
    cache = (x, W, b)
    return out, cache

## Affine backward
def affine_backward(dout, cache):
    """
    Inputs:
    - dout: Upstream derivative, of shape (N, M)
    - cache: Tuple of:
      - x: Input data, of shape (N, D)
      - w: Weights, of shape (D, M)

    Returns a tuple of:
    - dx: Gradient with respect to x, of shape (N, D)
    - dw: Gradient with respect to w, of shape (D, M)
    - db: Gradient with respect to b, of shape (M,)
    """    
    x, W, b = cache
    db = np.sum(dout, axis=0)
    dx = dout.dot(W.T).reshape(x.shape)
    dw = x.reshape(x.shape[0], -1).T.dot(dout)
    

### Softmax

In [15]:
def temporal_softmax_loss (y_s, y, mask, verbose=False):
    """
    A temporal version of softmax loss for use in RNNs. We assume that we are
    making predictions over a vocabulary of size M for each timestep of a
    timeseries of length T, over a minibatch of size N. The input y_s gives SCORES
    for all vocabulary elements at all timesteps, and y gives the INDICES of the
    ground-truth element at each timestep. We use a cross-entropy loss at each
    timestep, summing the loss over all timesteps and averaging across the minibatch.
    
    As an additional complication, we may want to ignore the model output at some
    timesteps, since sequences of different length may have been combined into a
    minibatch and padded with NULL tokens. The optional mask argument tells us
    which elements should contribute to the loss.
    
    Inputs:
    - y_s: Input scores, of shape (N, T, M)
    - y: Ground-truth indices, of shape (N, T) where each element is in the range
         0 <= y[i, t] < M
    - mask: Boolean array of shape (N, T) where mask[i, t] tells whether or not
      the scores at y_s[i, t] should contribute to the loss.
    Returns a tuple of:
    - loss: Scalar giving loss
    - dy_s: Gradient of loss with respect to scores y_s, of shape (N, T, M)
    """
    
    N, T, M = y_s.shape

    ys_flat = y_s.reshape(N * T, M)
    y_flat = y.reshape(N * T)
    mask_flat = mask.reshape(N * T)

    probs = np.exp(ys_flat - np.max(ys_flat, axis=1, keepdims=True))
    probs /= np.sum(probs, axis=1, keepdims=True)
    loss = -np.sum(mask_flat * np.log(probs[np.arange(N * T), y_flat])) / N
    dys_flat = probs.copy()
    dys_flat[np.arange(N * T), y_flat] -= 1
    dys_flat /= N
    dys_flat *= mask_flat[:, None]

    if verbose: print('dys_flat: ', dys_flat.shape)

    dy_s = dys_flat.reshape(N, T, M)

    return loss, dy_s
    

### Word embedding

In [16]:
def word_embedding_forward(x, W):
    """
    Forward pass for word embeddings. We operate on minibatches of size N where
    each sequence has length T. We assume a vocabulary of V words, assigning each
    to a vector of dimension D.
    Inputs:
    - x: Integer array of shape (N, T) giving indices of words. Each element idx
      of x muxt be in the range 0 <= idx < V.
    - W: Weight matrix of shape (V, D) giving word vectors for all words.
    Returns a tuple of:
    - out: Array of shape (N, T, D) giving word vectors for all input words.
    - cache: Values needed for the backward pass
    """
    out = W[x, :] 
    cache = (x, W)
    return out, cache


def word_embedding_backward(dout, cache):
    """
    Backward pass for word embeddings. We cannot back-propagate into the words
    since they are integers, so we only return gradient for the word embedding
    matrix.
    Inputs:
    - dout: Upstream gradients of shape (N, T, D)
    - cache: Values from the forward pass
    Returns:
    - dW: Gradient of word embedding matrix, of shape (V, D).
    """
    
    # x: (N, T)
    # W: (V, D)
    x, W = cache
    N, T, D = dout.shape

    dW = np.zeros_like(W)
    np.add.at(dW, x.reshape([-1]), dout.reshape([-1, D]))
    
    return dW

SyntaxError: 'return' outside function (<ipython-input-16-f0fd11ed0557>, line 39)

## Classifier

In [ ]:
#Initialisation
#Here, some vars are repeated to memorize

hidden_size = 200 # H, nb of neurons in hidden layer
seq_length = 25 # number of steps to unroll the RNN for <=> nb of chars putting in the input of RNN
learning_rate = 1e-1

type = "char"
data_size = len(char_data)
vocab_size = len(chars) # D
wordvec_size = vocab_size # V, if 'char' then D=V as we don't use wordvec

if type=="word":
    data_size = len(sents)
    vocab_size = len(word_to_index)
    wordvec_size = 128 

#init word vector
W_embed = np.random.randn(vocab_size, wordvec_size)
W_embed /=100
    
#init RNN parameters
W_xh = np.random.randn(wordvec_size, hidden_size) #(V,H)
W_xh /= wordvec_size
W_hh = np.random.randn(hidden_size, hidden_size) #(H,H)
W_hh /= hidden_size
W_hy = np.random.randn(hidden_size, vocab_size) #(H,M)
W_hy /= hidden_size
b_h = np.zeros((1, hidden_size)) #(H,)
b_y = np.zeros((1, vocab_size)) #(M,)

h_prev = np.zeros((1, hidden_size))

### Loss

In [18]:


    
def loss(text, img_fea=None, mode='train', type='char'):
    """
    Compute training-time loss.
    Inputs:
    - img_fea: Image features (used for captioning), of shape (N, D)
    - text: Ground-truth texts (e.g., captions); an integer array of shape (N, T) where
        each element is in the range 0 <= y[i, t] < M
    Returns:
        If training=True, then run a test-time forward pass of the model and return:
        - scores: Array of shape (N, M) giving classification scores, where
          scores[i, c] is the classification score for y[i] and the word/char with index c.

        Else, then run a training-time forward and backward pass and
        return a tuple of:
        - loss: Scalar value giving the loss
        - grads: Dictionary of gradients 
    """
    
    # Divide 'text' into two pieces 
    text_in = text[:, :-1]
    text_out = text[:, 1:]
    
    if type=='word':  mask = (text_out != unknown_token)
    else: mask = (text_out != unknown_char)
    
    
    ##### FORWARD STEPS ######
    embedded_text, cache_word_embedding = None, None
    
    # Embed the input word captions.
    if type=='word':
        embedded_text, cache_word_embedding = word_embedding_forward(text_in, W_embed)
    else: embedded_text = text_in
        
    #RNN forward
    h, y_s, cache_rnn = rnn_forward(embedded_text, h_prev, W_xh, W_hh, W_hy, b_h, b_y)
    
    if mode=='test':
        return y_s
    
    loss, grads = 0.0, {}

    #Softmax
    loss, dy_s = temporal_softmax_loss(y_s, text_out, mask)
    
    ##### BACKWARD STEPS ######
    
    #Backprop dy_s to get gradients
    dx, dh0, dWxh, dWhh, dWhy, dbh, dby = rnn_backward(dy_s, cache_rnn)
    grads['Wxh'], grads['Whh'], grads['Why'], grads['bh'], grads['by'] = dbh, dWxh, dWhh, dbh, dby

    # Backprop dx to get gradient for word embedding weights.
    if (type=='word'):
        dW_embed = word_embedding_backward(dx, cache_word_embedding)
        grads['W_embed'] = dW_embed
    
    return loss, grads
    

## Build model (classifier)

In [12]:
class LanguageModel

SyntaxError: invalid syntax (<ipython-input-12-72de117e8a38>, line 1)